In [14]:
from kafka import KafkaConsumer
from cassandra.cluster import Cluster
import json
import multiprocessing

In [15]:
kafka_bootstrap_severs = "192.168.56.1:9092"
kafka_topic = "logtrackingk"
cassandra_keyspace = 'study_de'

In [ ]:
def consume_from_kafka(cassandra_keyspace, kafka_topic, kafka_bootstrap_severs):
    cluster = Cluster()
    session = cluster.connect(cassandra_keyspace)
    consumer = KafkaConsumer(kafka_topic\
                             ,bootstrap_servers= kafka_bootstrap_severs\
                            , value_deserializer=lambda v: json.loads(v.decode('utf-8')))

    for message in consumer:
        data = message.value
        query = f""" INSERT INTO log_tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('{create_time}',{bid},{campaign_id},'{custom_track}',{group_id},{job_id},{publisher_id},'{ts}')"""
        print(query)
        session.execute(query)
        print("Reading data from kafka topic and write to cassandra done !!!")

In [16]:
cluster = Cluster()
session = cluster.connect(cassandra_keyspace)
consumer = KafkaConsumer(kafka_topic\
                            ,bootstrap_servers= kafka_bootstrap_severs\
                        , value_deserializer=lambda v: json.loads(v.decode('utf-8')))

for message in consumer:
    data = message.value
    query = f""" INSERT INTO log_tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('{create_time}',{bid},{campaign_id},'{custom_track}',{group_id},{job_id},{publisher_id},'{ts}')"""
    print(query)
    session.execute(query)
    print("Reading data from kafka topic and write to cassandra done !!!")

In [ ]:

process = multiprocessing.Process(target=consume_from_kafka, args= (cassandra_keyspace, kafka_topic, kafka_bootstrap_severs))
process.start()
process.join()

In [12]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

scala_version = '2.12'  # TODO: Ensure this is correct
spark_version = '3.4.0'
kafka_version = '3.5.0'

spark = SparkSession.builder\
   .appName("kafka-example")\
   .config("spark.jars", f'spark-sql-kafka-0-10_{scala_version}-{spark_version}.jar')\
   .config('spark.jars', f'kafka-clients-{kafka_version}.jar')\
   .config('spark.jars', 'org.apache.commons:commons-pool2:2.11.1.jar')\
   .getOrCreate()

In [13]:
df_read = spark\
  .read\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "192.168.56.1:9092")\
  .option("subscribe", "logtrackingk")\
  .load()
  
df_read.show(5)

+----+--------------------+------------+---------+------+--------------------+-------------+
| key|               value|       topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------------+---------+------+--------------------+-------------+
|null|[7B 22 63 72 65 6...|logtrackingk|        0|     0|2023-08-16 20:43:...|            0|
|null|[7B 22 63 72 65 6...|logtrackingk|        0|     1|2023-08-16 20:43:...|            0|
|null|[7B 22 63 72 65 6...|logtrackingk|        0|     2|2023-08-16 20:43:...|            0|
|null|[7B 22 63 72 65 6...|logtrackingk|        0|     3|2023-08-16 20:43:...|            0|
|null|[7B 22 63 72 65 6...|logtrackingk|        0|     4|2023-08-16 20:43:...|            0|
+----+--------------------+------------+---------+------+--------------------+-------------+
only showing top 5 rows

